In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv('test.csv')

In [5]:
print(df.iloc[1])

pixel0      0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Name: 1, Length: 784, dtype: int64


In [3]:
import torch
from torch import nn

In [4]:
class cnn(nn.Module):
    def __init__(self,numChannels,classes):
        super(cnn, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(in_channels=numChannels, out_channels=6,kernel_size=(5, 5),stride=(1,1),padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(1, 1),padding=1),
            nn.Conv2d(in_channels=6, out_channels=6,kernel_size=(2, 2),stride=(2,2),padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=6, out_channels=16,kernel_size=(5, 5),stride=(1,1),padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(1, 1),padding=1),
            nn.Conv2d(in_channels=16, out_channels=16,kernel_size=(2, 2),stride=(2,2),padding=0),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(in_features=256, out_features=120),
            nn.ReLU(),
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=classes)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
PATH = "cnn_model.pt"
model = cnn(1,10)
model.load_state_dict(torch.load(PATH))
model.eval()

cnn(
  (linear_relu_stack): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(1, 1), padding=1, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 6, kernel_size=(2, 2), stride=(2, 2))
    (4): ReLU()
    (5): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=(2, 2), stride=(1, 1), padding=1, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
    (9): ReLU()
    (10): Flatten(start_dim=1, end_dim=-1)
    (11): Linear(in_features=256, out_features=120, bias=True)
    (12): ReLU()
    (13): Linear(in_features=120, out_features=84, bias=True)
    (14): ReLU()
    (15): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [61]:
def filtered_img (x):
    x = x.reshape([28,28])
    non_zero_places = []
    binary = [0,1]
    for i in range(28):
        for j in range(28):
            if x[i,j] > 0:
                non_zero_places.append([i,j])
    for i in range(len(non_zero_places)):
        a = abs(non_zero_places[i][0] - non_zero_places[i-1][0])
        c = abs(non_zero_places[i][1] - non_zero_places[i-1][1])
        if i != len(non_zero_places)-1:
            d = abs(non_zero_places[i][1] - non_zero_places[i+1][1])
            b = abs(non_zero_places[i][0] - non_zero_places[i+1][0])
        else :
            b,d = 0,0
        if ((a > 3)or(b > 3)or(c > 3)or(d > 3)):
            x[non_zero_places[i][0],non_zero_places[i][1]] = 0
    return x

In [30]:
m = np.array(df.iloc[3]).reshape([28,28])
m[11,1] = 0
m[11,9] = 0

In [63]:
img = filtered_img(np.array(df.iloc[3])).reshape([1,1,28,28])*10
# print(img)
image = img.reshape([28,28])
# from matplotlib import pyplot as plt
# plt.imshow(img, interpolation='nearest')
# plt.show()

img = torch.tensor(img,dtype=torch.float)

In [64]:
model(img)

tensor([[ -8.8243, -16.3682,  -5.2371,  -2.8018,  -7.7351,  -6.3404, -13.2637,
          -1.5350,  -5.6310,   4.6478]], grad_fn=<AddmmBackward0>)

In [11]:
# import matplotlib.pyplot as plt

In [11]:
import cv2 as cv

In [65]:
cv.imwrite('number.png',image)

True

In [69]:
# from PIL import Image

In [66]:
img_ = cv.imread('number.png')

In [67]:
cv.imshow('number',img_)
cv.waitKey(0)

-1

In [17]:
'''
3
'''

'\n3\n'

In [18]:
df1 = pd.read_csv('train.csv')
y_train = df1["number"].iloc[0:41280]
x_train = df1.drop(columns='number').iloc[0:41280]
y_test = df1["number"].iloc[41280::]
x_test = df1.drop(columns='number').iloc[41280::]
loss_fn = nn.CrossEntropyLoss()

In [80]:
submissions = pd.read_csv('sample_submission.csv',index_col='ImageId')

In [81]:
df = pd.read_csv('test.csv')

In [82]:
X = torch.tensor(np.array(df).reshape([28000,1,28,28]),dtype=torch.float)*(10)
Y = model(X)

In [83]:
Y = Y.argmax(dim=1)
print(Y.shape)

torch.Size([28000])


In [84]:
Y = Y.numpy()
print(Y.shape)

(28000,)


In [85]:
for i in range (28000):
    submissions.iloc[i] = Y[i]

In [86]:
print(submissions)

         Label
ImageId       
1            2
2            0
3            9
4            9
5            3
...        ...
27996        9
27997        7
27998        3
27999        9
28000        2

[28000 rows x 1 columns]


In [88]:
submissions.to_csv('submission.csv')

In [19]:
size = 720
print(x_test.shape)
with torch.no_grad():
            X_test = torch.tensor(np.array(x_test).reshape([720,1,28,28]),dtype=torch.float)*(10)
            y_test = torch.tensor(np.array(y_test),dtype=torch.int64)
            Y_test = torch.nn.functional.one_hot(y_test, num_classes=10).type(torch.float)
            pred = model(X_test)
            test_loss = loss_fn(pred, Y_test).item()
            correct = (pred.argmax(1) == y_test).type(torch.float).sum().item()
            
test_loss /= size
correct /= size

print(f"Accuracy: {(100*correct)}%")

(720, 784)
Accuracy: 99.02777777777779%
